In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch
import xarray as xr
import numpy as np
import pandas as pd
import calendar
import seaborn as sns
import seaborn_image as isns

In [2]:
mesh_mask = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/masks/mesh_mask.nc')

In [3]:
mask = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/masks/tmaskm.nc')

In [4]:
# calculate volume for each grid cell

dx = mesh_mask.e1t 
dy = mesh_mask.e2t
grid_cell_area = dx * dy # m2

In [5]:
gridded_mesh = grid_cell_area.isel(t=0)

In [6]:
# apply mask to weighted grid cells

ocean_area = gridded_mesh * mask

In [7]:
ocean_area_sliced = ocean_area.sel(x=slice(520,595),y=slice(330,435),z=0,time=0)

In [8]:
ocean_area_sliced_dp = ocean_area_sliced.isel(x=slice(58,60),y=slice(41,43))

# European average

### Alkalinity baseline

In [9]:
alk_weighted_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/alkalinity_base_26.nc', decode_times=True)
mld_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/mld_base_26.nc')

In [10]:
alk_weighted_baseline = alk_weighted_baseline.sel(time_counter=slice('2090','2100'))

In [11]:
mld_baseline = mld_baseline.sel(time_counter=slice('2090','2100'))

In [12]:
mld = alk_weighted_baseline.deptht < mld_baseline.somxl010

In [13]:
mld = mld.to_dataset(name='mld') 

In [14]:
alk_base = alk_weighted_baseline * mld.mld

In [15]:
alk_base = alk_base.where(alk_base)

In [16]:
alk_base = alk_base.fillna(0)

In [17]:
alk_base = alk_base * 1.025 # mmol/m3

In [18]:
# iterate over all layers and append

alk_base_layers = []

for x in list((range(len(alk_base.deptht)))):
    if x == 0:
        alk_n_base = alk_base.ALK.isel(deptht=x) * alk_base.deptht[x] 
        alk_base_layers.append(alk_n_base)
    else:
        alk_n_base = alk_base.ALK.isel(deptht=x) * (alk_base.deptht[x] - alk_base.deptht[x-1])
        alk_base_layers.append(alk_n_base)

In [19]:
weighted_base_alk = sum(alk_base_layers) / mld_baseline.somxl010 # mmol/m3 * m / m

In [20]:
weighted_base_alk = weighted_base_alk.to_dataset(name='ALK')

In [21]:
weighted_base_alk.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/mld_alk/weighted_base_alk.nc')

In [22]:
sliced_alk_baseline = weighted_base_alk.ALK * ocean_area_sliced  # baseline

In [23]:
regridded_alk_base = sliced_alk_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [24]:
regridded_alk_base = regridded_alk_base.tmask.rename("ALK")

In [25]:
regridded_alk_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/mld_alk/regridded_alk_base.nc')

### Alkalinity OAE

In [26]:
alk_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/alkalinity_oae_26.nc')
mld_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/mld_oae_26.nc')

In [27]:
alk_weighted_oae = alk_weighted_oae.sel(time_counter=slice('2090','2100'))

In [28]:
mld_oae = mld_oae.sel(time_counter=slice('2090','2100'))

In [29]:
mld = alk_weighted_oae.deptht < mld_oae.somxl010

In [30]:
mld = mld.to_dataset(name='mld') 

In [31]:
alk_oae = alk_weighted_oae * mld.mld

In [32]:
alk_oae = alk_oae.where(alk_oae)

In [33]:
alk_oae = alk_oae.fillna(0)

In [34]:
alk_oae = alk_oae * 1.025 # mmol/m3

In [35]:
# iterate over all layers and append

alk_oae_layers = []

for x in list((range(len(alk_oae.deptht)))):
    if x == 0:
        alk_n_oae = alk_oae.ALK.isel(deptht=x) * alk_oae.deptht[x] 
        alk_oae_layers.append(alk_n_oae)
    else:
        alk_n_oae = alk_oae.ALK.isel(deptht=x) * (alk_oae.deptht[x] - alk_oae.deptht[x-1])
        alk_oae_layers.append(alk_n_oae)

In [36]:
weighted_oae_alk = sum(alk_oae_layers) / mld_oae.somxl010 # mmol/m3 * m / m

In [37]:
weighted_oae_alk = weighted_oae_alk.to_dataset(name='ALK')

In [38]:
weighted_oae_alk.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/mld_alk/weighted_oae_alk.nc')

In [39]:
sliced_alk_oae = ocean_area_sliced * weighted_oae_alk.ALK # baseline

In [40]:
regridded_alk_oae = sliced_alk_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [41]:
regridded_alk_oae = regridded_alk_oae.tmask.rename("ALK")

In [42]:
regridded_alk_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/mld_alk/regridded_alk_oae.nc')

### DIC baseline

In [43]:
dic_weighted_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/dic_base_26.nc')
mld_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/mld_base_26.nc')

In [44]:
dic_weighted_baseline = dic_weighted_baseline.sel(time_counter=slice('2090','2100'))

In [45]:
mld_baseline = mld_baseline.sel(time_counter=slice('2090','2100'))

In [46]:
mld = dic_weighted_baseline.deptht < mld_baseline.somxl010

In [47]:
mld = mld.to_dataset(name='mld') 

In [48]:
dic_base = dic_weighted_baseline * mld.mld

In [49]:
dic_base = dic_base.where(dic_base)

In [50]:
dic_base = dic_base.fillna(0)

In [51]:
dic_base = dic_base * 1.025 # mmol/m3

In [52]:
# iterate over all layers and append

dic_base_layers = []

for x in list((range(len(dic_base.deptht)))):
    if x == 0:
        dic_n_base = dic_base.DIC.isel(deptht=x) * dic_base.deptht[x] 
        dic_base_layers.append(dic_n_base)
    else:
        dic_n_base = dic_base.DIC.isel(deptht=x) * (dic_base.deptht[x] - dic_base.deptht[x-1])
        dic_base_layers.append(dic_n_base)

In [53]:
weighted_base_dic = sum(dic_base_layers) / mld_baseline.somxl010 # mmol/m3 * m / m

In [54]:
weighted_base_dic = weighted_base_dic.to_dataset(name='DIC')

In [55]:
weighted_base_dic.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/dic_mld//weighted_base_dic_2.6.nc')

In [56]:
sliced_dic_baseline = ocean_area_sliced * weighted_base_dic.DIC # baseline

In [57]:
regridded_dic_base = sliced_dic_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [58]:
regridded_dic_base = regridded_dic_base.tmask.rename("DIC")

In [59]:
regridded_dic_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/dic_mld/regridded_dic_base_2.6.nc')

### DIC OAE

In [60]:
mld_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/mld_oae_26.nc')

In [61]:
dic_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/dic_oae_26.nc', decode_times=True)

In [62]:
dic_weighted_oae = dic_weighted_oae.sel(time_counter=slice('2090','2100'))

In [63]:
mld_oae = mld_oae.sel(time_counter=slice('2090','2100'))

In [64]:
mld = dic_weighted_oae.deptht < mld_oae.somxl010

In [65]:
mld = mld.to_dataset(name='mld') 

In [66]:
dic_oae = dic_weighted_oae * mld.mld

In [67]:
dic_oae = dic_oae.where(dic_oae)

In [68]:
dic_oae = dic_oae.fillna(0)

In [69]:
dic_oae = dic_oae * 1.025 # mmol/m3

In [70]:
# iterate over all layers and append

dic_oae_layers = []

for x in list((range(len(dic_oae.deptht)))):
    if x == 0:
        dic_n_oae = dic_oae.DIC.isel(deptht=x) * dic_oae.deptht[x] 
        dic_oae_layers.append(dic_n_oae)
    else:
        dic_n_oae = dic_oae.DIC.isel(deptht=x) * (dic_oae.deptht[x] - dic_oae.deptht[x-1])
        dic_oae_layers.append(dic_n_oae)

In [71]:
weighted_oae_dic = sum(dic_oae_layers) / mld_oae.somxl010 # mmol/m3 * m / m

In [72]:
weighted_oae_dic = weighted_oae_dic.to_dataset(name='DIC')

In [73]:
weighted_oae_dic.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/dic_mld//weighted_oae_dic_2.6.nc')

In [74]:
sliced_dic_oae = ocean_area_sliced * weighted_oae_dic.DIC # baseline

In [75]:
regridded_dic_oae = sliced_dic_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [76]:
regridded_dic_oae = regridded_dic_oae.tmask.rename("DIC")

In [77]:
regridded_dic_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/dic_mld//regridded_dic_oae_2.6.nc')

# Average at Location S 

### Alkalinity baseline

In [78]:
alk_weighted_baseline_dp = alk_weighted_baseline.ALK.isel(x=slice(58,60),y=slice(41,43))

In [79]:
mld_baseline_dp = mld_baseline.somxl010.isel(x=slice(58,60),y=slice(41,43))

In [80]:
alk_weighted_baseline_dp = alk_weighted_baseline_dp.to_dataset(name='ALK')

In [81]:
mld_baseline_dp = mld_baseline_dp.to_dataset(name='mld')

In [82]:
mld_dp = alk_weighted_baseline_dp.deptht < mld_baseline_dp.mld

In [83]:
mld_dp = mld_dp.to_dataset(name='mld') 

In [84]:
alk_base_dp = alk_weighted_baseline_dp * mld_dp.mld

In [85]:
alk_base_dp = alk_base_dp.where(alk_base_dp)

In [86]:
alk_base_dp = alk_base_dp.fillna(0)

In [87]:
alk_base_dp = alk_base_dp * 1.025 # mmol/m3

In [88]:
# iterate over all layers and append

alk_base_layers_dp = []

for x in list((range(len(alk_base_dp.deptht)))):
    if x == 0:
        alk_n_base_dp = alk_base_dp.ALK.isel(deptht=x) * alk_base_dp.deptht[x] 
        alk_base_layers_dp.append(alk_n_base_dp)
    else:
        alk_n_base_dp = alk_base_dp.ALK.isel(deptht=x) * (alk_base_dp.deptht[x] - alk_base_dp.deptht[x-1])
        alk_base_layers_dp.append(alk_n_base_dp)

In [89]:
weighted_base_alk_dp = sum(alk_base_layers_dp) / mld_baseline_dp.mld # mmol/m3 * m / m

In [90]:
weighted_base_alk_dp = weighted_base_alk_dp.to_dataset(name='ALK')

In [91]:
sliced_alk_baseline_dp = ocean_area_sliced_dp * weighted_base_alk_dp.ALK # baseline

In [92]:
regridded_alk_base_dp = sliced_alk_baseline_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [93]:
regridded_alk_base_dp = regridded_alk_base_dp.tmask.rename("ALK")

In [94]:
regridded_alk_base_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/mld_alk/regridded_alk_base_dp.nc')

### Alkalinity OAE

In [95]:
alk_weighted_oae_dp = alk_weighted_oae.ALK.isel(x=slice(58,60),y=slice(41,43))

In [96]:
mld_oae_dp = mld_oae.somxl010.isel(x=slice(58,60),y=slice(41,43))

In [97]:
alk_weighted_oae_dp = alk_weighted_oae_dp.to_dataset(name='ALK')

In [98]:
mld_oae_dp = mld_oae_dp.to_dataset(name='mld')

In [99]:
mld_dp = alk_weighted_oae_dp.deptht < mld_oae_dp.mld

In [100]:
mld_dp = mld_dp.to_dataset(name='mld') 

In [101]:
alk_oae_dp = alk_weighted_oae_dp * mld_dp.mld

In [102]:
alk_oae_dp = alk_oae_dp.where(alk_oae_dp)

In [103]:
alk_oae_dp = alk_oae_dp.fillna(0)

In [104]:
alk_oae_dp = alk_oae_dp * 1.025 # mmol/m3

In [105]:
# iterate over all layers and append

alk_oae_layers_dp = []

for x in list((range(len(alk_oae_dp.deptht)))):
    if x == 0:
        alk_n_oae_dp = alk_oae_dp.ALK.isel(deptht=x) * alk_oae_dp.deptht[x] 
        alk_oae_layers_dp.append(alk_n_oae_dp)
    else:
        alk_n_oae_dp = alk_oae_dp.ALK.isel(deptht=x) * (alk_oae_dp.deptht[x] - alk_oae_dp.deptht[x-1])
        alk_oae_layers_dp.append(alk_n_oae_dp)

In [106]:
weighted_oae_alk_dp = sum(alk_oae_layers_dp) / mld_oae_dp.mld # mmol/m3 * m / m

In [107]:
weighted_oae_alk_dp = weighted_oae_alk_dp.to_dataset(name='ALK')

In [108]:
sliced_alk_oae_dp = ocean_area_sliced_dp * weighted_oae_alk_dp.ALK # baseline

In [109]:
regridded_alk_oae_dp = sliced_alk_oae_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [110]:
regridded_alk_oae_dp = regridded_alk_oae_dp.tmask.rename("ALK")

In [111]:
regridded_alk_oae_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/mld_alk/regridded_alk_oae_dp.nc')

### DIC baseline

In [112]:
dic_weighted_baseline_dp = dic_weighted_baseline.DIC.isel(x=slice(58,60),y=slice(41,43))

In [116]:
mld_baseline_dp = mld_baseline.somxl010.isel(x=slice(58,60),y=slice(41,43))

In [117]:
dic_weighted_baseline_dp = dic_weighted_baseline_dp.to_dataset(name='DIC')

In [118]:
mld_baseline_dp = mld_baseline_dp.to_dataset(name='mld')

In [119]:
mld_dp = dic_weighted_baseline_dp.deptht < mld_baseline_dp.mld

In [120]:
mld_dp = mld_dp.to_dataset(name='mld') 

In [121]:
dic_base_dp = dic_weighted_baseline_dp * mld_dp.mld

In [122]:
dic_base_dp = dic_base_dp.where(dic_base_dp)

In [123]:
dic_base_dp = dic_base_dp.fillna(0)

In [124]:
dic_base_dp = dic_base_dp * 1.025 # mmol/m3

In [125]:
# iterate over all layers and append

dic_base_layers_dp = []

for x in list((range(len(dic_base_dp.deptht)))):
    if x == 0:
        dic_n_base_dp = dic_base_dp.DIC.isel(deptht=x) * dic_base_dp.deptht[x] 
        dic_base_layers_dp.append(dic_n_base_dp)
    else:
        dic_n_base_dp = dic_base_dp.DIC.isel(deptht=x) * (dic_base_dp.deptht[x] - dic_base_dp.deptht[x-1])
        dic_base_layers_dp.append(dic_n_base_dp)

In [126]:
weighted_base_dic_dp = sum(dic_base_layers_dp) / mld_baseline_dp.mld # mmol/m3 * m / m

In [127]:
weighted_base_dic_dp = weighted_base_dic_dp.to_dataset(name='DIC')

In [128]:
sliced_dic_baseline_dp = ocean_area_sliced_dp * weighted_base_dic_dp.DIC # baseline

In [129]:
regridded_dic_base_dp = sliced_dic_baseline_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [130]:
regridded_dic_base_dp = regridded_dic_base_dp.tmask.rename("DIC")

In [131]:
regridded_dic_base_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/dic_mld/regridded_dic_base_dp.nc')

### DIC OAE

In [140]:
dic_weighted_oae_dp = dic_weighted_oae.DIC.isel(x=slice(58,60),y=slice(41,43))

In [141]:
mld_oae_dp = mld_oae.somxl010.isel(x=slice(58,60),y=slice(41,43))

In [142]:
dic_weighted_oae_dp = dic_weighted_oae_dp.to_dataset(name='DIC')

In [143]:
mld_oae_dp = mld_oae_dp.to_dataset(name='mld')

In [144]:
mld_dp = dic_weighted_oae_dp.deptht < mld_oae_dp.mld

In [145]:
mld_dp = mld_dp.to_dataset(name='mld') 

In [146]:
dic_oae_dp = dic_weighted_oae_dp * mld_dp.mld

In [147]:
dic_oae_dp = dic_oae_dp.where(dic_oae_dp)

In [148]:
dic_oae_dp = dic_oae_dp.fillna(0)

In [149]:
dic_oae_dp = dic_oae_dp * 1.025 # mmol/m3

In [150]:
# iterate over all layers and append

dic_oae_layers_dp = []

for x in list((range(len(dic_oae_dp.deptht)))):
    if x == 0:
        dic_n_oae_dp = dic_oae_dp.DIC.isel(deptht=x) * dic_oae_dp.deptht[x] 
        dic_oae_layers_dp.append(dic_n_oae_dp)
    else:
        dic_n_oae_dp = dic_oae_dp.DIC.isel(deptht=x) * (dic_oae_dp.deptht[x] - dic_oae_dp.deptht[x-1])
        dic_oae_layers_dp.append(dic_n_oae_dp)

In [151]:
weighted_oae_dic_dp = sum(dic_oae_layers_dp) / mld_oae_dp.mld # mmol/m3 * m / m

In [152]:
weighted_oae_dic_dp = weighted_oae_dic_dp.to_dataset(name='DIC')

In [153]:
sliced_dic_oae_dp = ocean_area_sliced_dp * weighted_oae_dic_dp.DIC # baseline

In [154]:
regridded_dic_oae_dp = sliced_dic_oae_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [155]:
regridded_dic_oae_dp = regridded_dic_oae_dp.tmask.rename("DIC")

In [156]:
regridded_dic_oae_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/dic_mld/regridded_dic_oae_dp.nc')